In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fpath

from lib.gallerosalas.preprocess_aud_raw import preprocess
import lib.gallerosalas.preprocess_common as prepcommon


%load_ext autoreload
%autoreload 2

In [ ]:
FPS = 20  # Hz
mice = ['mou_9', 'mou_6', 'mou_7', 'mou_5']
extra_mice = ['mou_15_Ariel', 'mou_18_Ariel']

In [ ]:
pathPrefix = '/run/user/1000/gvfs/smb-share:server=130.60.51.15,share=neurophysiology-storage2/'

In [ ]:
pathAUD = pathPrefix + 'Gallero/DATA PER MICE (raw and preprocessed)/Widefield_imaging mice/'

In [ ]:
# pathOverlay = gui_fpath('root path to overlay')
# pathOverlay = pathPrefix + 'SDT-TDT/overlay'
pathOverlay = pathPrefix + 'Gallero/Data_processed/SDT-TDT/overlay'

In [ ]:
# pathPreferences = gui_fpath('root path to overlay')
pathPreferences = '/media/alyosha/Data/TE_data/yasirdata_aud_raw'
# pathPreferences = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_aud_raw'

In [ ]:
pathDict = {
    'AUD' : pathAUD,
    'Overlay' : pathOverlay,
    'Preferences' : pathPreferences
}

In [ ]:
prep = preprocess(pathDict)

# 1. Test transform on Ref files

Not necessary because already done in texture

# 2. Test transform on Video Files

In [ ]:
%%time
for mousename in sorted(set(prep.dataPaths['mouse'])):
    print(mousename)
    prep.test_transform_vids(mousename)

# 3. Pool-process video files

In [ ]:
prep.process_video_files('mou_5', skipExisting=False)

## 3.1 Test if result is sensible

In [ ]:
import h5py 
rez = []
for idx, row in prep.dataPaths.iterrows():
    session = row['day'] + '_' + row['session']
    fpath = os.path.join(pathPreferences, row['mouse'] + '.h5')    
    
    with h5py.File(fpath, 'r') as f:        
        dataRSP = np.copy(f['data'][session])
        rez += [np.nanmean(dataRSP, axis=(0, 1))]
        
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.DataFrame(np.array(rez), columns=np.arange(27)))

# 4. Pull trial types and times

* Also compute d'

In [ ]:
import h5py

for mouseName in mice:
    with h5py.File(os.path.join(pathPreferences, mouseName + '.h5'), 'a') as f:
        for key in ['metadata', 'accuracy', 'dprime']:
            if key in f.keys():
                print(key)
                del f[key]

In [ ]:
prep.process_metadata_files(pathPreferences)

In [ ]:
for idx, row in prep.dataPaths.iterrows():
    session = row['day'] + '_' + row['session']
    fpath = os.path.join(pathPreferences, row['mouse'] + '.h5')
    with h5py.File(fpath) as f:
        print(fpath, f.keys())
    
    df = pd.read_hdf(fpath, '/metadata/' + session)
    display(df.head())

In [ ]:
prep.extract_timestamps_video(pathPreferences)

# 5. Cleanup bad sessions/trials

In [ ]:
prep.metadata_add_selected_column(pathPreferences)

In [ ]:
badIdxs = prep.get_trial_idxs_by_timestep_interval(pathPreferences, 'mou_5', '2016_12_07_session02', 43000, 100000)
print(badIdxs)

In [ ]:
prep.mark_trials_bad_by_index(pathPreferences, 'mou_5', '2016_12_07_session02', badIdxs)

# 6. Baseline subtraction

# 6.1 Plot stitched session, fit poly

**TODO-This**:
* Test if there are trials of mismatching length
* Test if there are sessions with mismatching counts, process
* 

**TODO**:
* Purge mismatching trial counts for trialStruct vs vid:
    - Go over data, delete last
* Very high variance of individual consecutive trials. May render entire idea useless
  - Explore more channels, mice, naive vs expert
  - Compare if Yaro has the same problem
  - Ask Yasir what he thinks about the origin of these fluctuations:
      - Real?
      - Motion artifacts?
      - Sth else?

In [ ]:
## Testing individual fitness
%matplotlib notebook
times, dataRSP = prep.get_pooled_data_rel_times(pathPreferences, 'mou_5', "2016_12_08_session05", onlySelected=True)
prepcommon.example_poly_fit(times, dataRSP, ord=15, iCh=12)

In [ ]:
## Testing individual fitness
%matplotlib inline
for idx, row in prep.dataPaths.iterrows():
    try: 
        session = row['day'] + '_' + row['session']
#         session="2017_03_29_session01"
        print(row['mouse'], session)
    
        times, dataRSP = prep.get_pooled_data_rel_times(pathPreferences, row['mouse'], session)
        
        for iCh in range(dataRSP.shape[2]):
            prepcommon.example_poly_fit(times, dataRSP, ord=15, iCh=iCh)
            break
    except:
        print('-- Failed')
    break

# 6.2 Process all sessions

In [ ]:
FPS = 20
iMin = 0 * FPS
iMax = 1 * FPS

In [ ]:
prep.baseline_subtraction_dff(pathPreferences, iMin, iMax, skipExist=True)

In [ ]:
prep.baseline_subtraction_poly(pathPreferences, ord=15, skipExist=True)

# 7. Cleanup - session names

All sessions should have unique labels for future analysis. Since this is not the case, we must append mousename as prefix to all sessions

In [ ]:
import h5py

In [ ]:
for mousename in mice:
    pwdH5 = os.path.join(pathPreferences, mousename+'.h5')
    with h5py.File(pwdH5, 'a') as f:
        fields = set(f.keys())
        for field in fields:
            sessions = list(f[field].keys())
            for session in sessions:
                if mousename in session:
                    print('--', field, session, 'already renamed')
                else:
                    print('--', field, session, 'renaming to', sNew)
                    sNew = mousename + '_' + session
                    
                    if sNew in sessions:
                        print('---', field, session, 'also deleting existing', sNew)
                        del f[field][session]
                        
                    f.move('/metadata/'+session, '/metadata/'+sNew)

# 8. Sanity checking

## 8.1 Is reward part of video

Yes for all except MOU6

In [ ]:
prep.check_reward_in_data(pathPreferences)

# 9. Behaviour pooling

## 9.1 Movement Vectors

In [ ]:
prep.dataPaths

In [ ]:
sig2 = (0.05)**2 / 16
prep.behaviour_tune_resample_kernel('mou_9', '2017_12_12_session04', sig2,
                                    trialType='moveVect_CR', trialIdx=4, srcFreq=30.0, trgFreq=20.0)

In [ ]:
prep.read_parse_behaviour(pathPreferences, skipExisting=False)